In [1]:
from transformer import Transformer
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)
print("Connecting to TPU...")
resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu='node-8',zone='us-central1-f')
strategy = tf.distribute.TPUStrategy(resolver)
print("Done!")
print("Number of accelerators: ", strategy.num_replicas_in_sync)

TensorFlow version:  2.12.0
Connecting to TPU...
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-8


INFO:tensorflow:Initializing the TPU system: node-8


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


Done!
Number of accelerators:  8


In [2]:
from transformers import GPT2Tokenizer
tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

saved_transformer_path = 'gs://saved_transformers/v2'
with strategy.scope():
    num_layers = 2
    d_model = 192
    num_heads = 8
    dff = 1024
    input_vocab_size = tokenizer.vocab_size
    target_vocab_size = tokenizer.vocab_size
    max_position_encoding = 1000
    dropout_rate = 0.1
    learning_rate = 1e-3
    batch_size = 64
    epochs = 3
    transformer = Transformer(num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, max_position_encoding, dropout_rate)
    tf.saved_model.load(saved_transformer_path, options=tf.saved_model.LoadOptions(experimental_io_device='/job:localhost'), tags=None, custom_objects={'Transformer': Transformer})

In [3]:
# Example usage
def predict_next_word(input_text, transformer, tokenizer, top_k=5):
    input_tokens = tokenizer.encode(input_text, return_tensors="tf")
    logits = transformer([input_tokens, tf.zeros_like(input_tokens)], training=False)
    logits = logits[:, -1, :]  # Get the logits for the last token
    top_k_indices = tf.math.top_k(logits, k=top_k).indices
    top_k_tokens = [tokenizer.decode([token_id]) for token_id in top_k_indices.numpy()[0]]
    
    return top_k_tokens

input_text = """
The first political philosopher to call himself an anarchist (French: anarchiste) was Pierre-"""
predicted_words = predict_next_word(input_text, transformer, tokenizer, top_k=5)
print(f"Input: {input_text}")
print("Predicted next words:")
for i, word in enumerate(predicted_words):
    print(f"{i + 1}. {word}")

Input: 
The first political philosopher to call himself an anarchist (French: anarchiste) was Pierre-
Predicted next words:
1. flying
2. �
3. abase
4.  contingent
5.  merch
